In [37]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from helper_funcs import *
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.ensemble import GradientBoostingClassifier


In [2]:
RF = get_all()

----pulling Redfin data from Azure storage----
----Done----
----pulling schools data from Azure storage----
----Done----
----merging all data----
----Done----
Shape: (75360, 22)


In [15]:
## Create Linear Dataframe


RF = linear[linear['Sale_Type'] == 'MLS Listing']

In [18]:
def get_linear(RF):
    RF.drop(RF[RF.BEDS.isna() | RF.BATHS.isna()].index, inplace=True)
    RF.loc[RF.overallRating.isna(),'overallRating'] = 5
    ### Split data into train test split
    #pd.set_option('display.max_rows', None)
    zip_houses = RF.groupby('zip').agg('count')['PRICE'].to_dict()
    RF['houses_perZIP'] = RF.zip.apply(lambda r: zip_houses[r])
    RF = RF[RF.houses_perZIP>4]
    # Use groupby to split the df into smaller dfs for each ZIP
    groups = RF.groupby('zip')
    dfs = [groups.get_group(x) for x in groups.groups]
    # Use first df to initiate X_train, X_test, etc
    for d in dfs[:1]:
        feat = d[['Prop_Type','BEDS','BATHS','SF','Lot_Size','YearBuilt','zip']]
        feat.zip = feat.zip.astype('object')
        y = np.log10(d.PRICE)
        X_train, X_test, y_train, y_test = train_test_split(feat, y)

    # train_test_split each df then stack together
    for d in dfs[1:]:
        feat = d[['Prop_Type','BEDS','BATHS','SF','Lot_Size','YearBuilt','zip']]
        feat.zip = feat.zip.astype('object')
        y = np.log10(d.PRICE)
        X_trainpiece, X_testpiece, y_trainpiece, y_testpiece = train_test_split(feat, y, test_size=0.25)
        X_train = X_train.append(X_trainpiece)
        X_test = X_test.append(X_testpiece)
        y_train = y_train.append(y_trainpiece)
        y_test = y_test.append(y_testpiece)
        
    X_train_linear = pd.get_dummies(X_train, drop_first=True)
    X_test_linear = pd.get_dummies(X_test, drop_first=True)
    return X_train_linear, X_test_linear, y_train, y_test


In [26]:
def get_tree(RF):
    RF.drop(RF[RF.BEDS.isna() | RF.BATHS.isna()].index, inplace=True)
    RF.loc[RF.overallRating.isna(),'overallRating'] = 5
    ### Split data into train test split
    #pd.set_option('display.max_rows', None)
    zip_houses = RF.groupby('zip').agg('count')['PRICE'].to_dict()
    RF['houses_perZIP'] = RF.zip.apply(lambda r: zip_houses[r])
    RF = RF[RF.houses_perZIP>4]
    # Use groupby to split the df into smaller dfs for each ZIP
    groups = RF.groupby('zip')
    dfs = [groups.get_group(x) for x in groups.groups]
    # Use first df to initiate X_train, X_test, etc
    for d in dfs[:1]:
        feat = d[['Prop_Type','BEDS','BATHS','SF','Lot_Size','YearBuilt','zip']]
        feat.zip = feat.zip.astype('object')
        y = np.log10(d.PRICE)
        X_train, X_test, y_train, y_test = train_test_split(feat, y)

    # train_test_split each df then stack together
    for d in dfs[1:]:
        feat = d[['Prop_Type','BEDS','BATHS','SF','Lot_Size','YearBuilt','zip']]
        feat.zip = feat.zip.astype('object')
        y = np.log10(d.PRICE)
        X_trainpiece, X_testpiece, y_trainpiece, y_testpiece = train_test_split(feat, y, test_size=0.25)
        X_train = X_train.append(X_trainpiece)
        X_test = X_test.append(X_testpiece)
        y_train = y_train.append(y_trainpiece)
        y_test = y_test.append(y_testpiece)

    X_train_le = X_train.copy()
    X_test_le = X_test.copy()
    le = LabelEncoder()
    X_train_le.Prop_Type = le.fit_transform(X_train_le.Prop_Type)
    X_test_le.Prop_Type = le.fit_transform(X_test_le.Prop_Type)
    return X_train_le, X_test_le, y_train, y_test

In [19]:
X_train_linear, X_test_linear, y_train, y_test = get_linear(RF)

In [27]:
X_train_tree, X_test_tree, y_train, y_test = get_tree(RF)

In [ ]:
## Testing out Models

In [35]:
### Linear Regression
lm = LinearRegression()
lm.fit(X_train_linear, y_train)
print(f'train R2: {lm.score(X_train_linear, y_train)}')
print(f'train R2: {lm.score(X_test_linear, y_test)}')

train R2: 0.40657685038589264
train R2: 0.3591230416941972


In [34]:
## CatBoost

cat = CatBoostRegressor(verbose=False)
cat.get_params()
cat.fit(X_train_tree, y_train)
print(f'Train R2: {cat.score(X_train_tree, y_train)}')
print(f'Test R2: {cat.score(X_test_tree, y_test)}')

Train R2: 0.8246070251399195
Test R2: 0.7958135932320387
